In [1]:
from PIL import Image, ImageDraw, ImageFont, ImageFilter
import glob
import numpy as np
import os
import random
import imgaug as ia
from imgaug import augmenters as iaa
import cv2
from zhon import hanzi
import glob

ModuleNotFoundError: No module named 'zhon'

## 预备知识
* 文件读取
* PIL使用
* os操作

### 文件读取
`f.read(-1)`读取指定大小的字符串  
`f.readline()`读取一行内容  
`f.readlines()`读取所有行的内容，返回一个包含所有行的list

### glob
glob模块支持的通配符：

通配符	功能   
\*	匹配0或多个字符  
**	匹配所有文件、目录、子目录和子目录里的文件（3.5版本新增）  
?	匹配1个字符，与正则表达式里的?不同   
[exp]	匹配指定范围内的字符，如：[1-9]匹配1至9范围内的字符  
[!exp]	匹配不在指定范围内的字符  


In [80]:
os.path.splitext('dasf/asdf.sd')

('dasf/asdf', '.sd')

In [4]:

row_collections = glob.glob("手写字体数据集/"+"TL*")#匹配行段落


In [81]:

def crop_img(img):
    """
    Return:(y1, x1, y2, x2)最大外接矩形的左上坐标和右下坐标
    """
    h, w = img.shape
    margin = np.random.randint(10, 20)#留出一点的空白部分,否则会导致识别出问题
    horiziontal_indicies = np.where(np.any(img, axis=0))[0]
    vertival_indicies = np.where(np.any(img, axis=1))[0]
    if horiziontal_indicies.shape[0]:
        x1, x2 = horiziontal_indicies[[0, -1]]
        y1, y2 = vertival_indicies[[0, -1]]
#         x2 += 1
#         y2 += 1
    else:
        x1, x2, y1, y2 = 0, 0, 0, 0
    x1 = max(x1 - margin, 0)#最大最小原则
    x2 = min(x2 + margin, w)
    return 255 - img[y1:y2, x1:x2] #直接返回crop好的图片
a = np.array([[0,0,1,1,0,0],[0,1,1,1,1,0],[0,0,0,1,0,0],[0,0,0,0,0,0]])
# print(a)
extract_outbox(a)
sample = Image.open("hand_sample.bmp").convert("L")#bmp使用灰度读取
print(sample.size)
# sample.show()
sample_arr = np.array(sample)
sample_arr = 255 -sample_arr
bbox = crop_img(sample_arr)
print(bbox.shape)
newCanvas = Image.new("RGB", size=(2500, 2500), color='white')
Image.fromarray(bbox).show()
draw = ImageDraw.Draw(sample)
draw.line(100, 200)
# draw.rectangle(bbox, outline=2)

sample.show()
# newCanvas.show()
# print(sample_arr)
# bbox = extract_outbox(sample_arr)
# # sample_arr[bbox[0]:bbox[2], bbox[1]:bbox[3]].astype(np.int32)
# # Image.fromarray(sample_arr).show()

(1671, 108)
(103, 1478)


TypeError: argument must be sequence

注意list和array不是一个东西  
list中可能有很多个对象,但是array就只是一个对象,可能二者里面包含的数值都是一样的,但是用法上还是不能混同

>注意,如果Image对象是灰度图时,传入的所有颜色color参数只能有一个通道,如果传入元组会报错!!!!

In [29]:
ImageDraw.ImageDraw.rectangle?

In [192]:
np.maximum(1, [1,2,3,4,5])

array([1, 2, 3, 4, 5])

In [193]:
np.maximum?

In [ ]:
def 

`np.any(axis=)`axis=0沿着另一个轴


### Image
该基类是`Image.Image`,如果需要help在这个类的基础上查看  
**方法和属性：**
`Image.split()=>多个通道的list`通道分离,  
`Image.resize((size))`图像尺寸变化  
`Image.convert("RGB")`图像色彩模式转换

### ImageDraw
`ImageDraw.ImageDraw`这是父类，下面的方法都是基于这个类，如果需要help用这个后面加方法或属性
PIL画盘，需要使用该函数将待处理图像放入Draw(Image)，返回一个句柄对象用来作画  
**常用方法介绍：**  
`ImageDraw.Draw()`画盘将图像包住  
`ImageDraw.textsize(text="content", font=fontObj)=>(width, height)`返回给定字符串的分辨率大小，以像素为单位  

`ImageDraw.text([loc], text="content", font=fontObj, fill=(color,color,color))`写文字  
`ImageFont.truetype(font=font_type, size=font_size)`设定字体,font传入字体路径    
`draw.rectangle([xy], outline=(color), fill=(color))`
功能：绘制一个长边形。
变量box是包含2元组[(x,y),…]或者数字[x,y,…]的任何序列对象。它应该包括2个坐标值(左上坐标axis=1和右下坐标axis=0),  
注意：当长方形没有没有被填充时，第二个坐标对定义了一个长方形外面的点。  
变量options可以绘制外框和矩形色块，**outline是指定外框的颜色**，fill是指定矩形色块填充颜色  

`Image.paste(img, loc)`将img粘贴到self上
* 粘贴新图片至图片中，loc 参数可以为 2-元组（upper, left）只需要传左上坐标就好了，或是 4-元组（left, upper, right, lower），或者是 None  
* 功能同上。不过是将指定位置填充为某种颜色。

In [133]:
np.where?

In [134]:
def make_dirs(path):
    if not os.path.exists(path):
        os.makedirs(path)
        
        
background_imgs_dir = "bg_img/"
fonts_dir = "fonts/"
handwriting_imgs_dir = "手写字体数据集/"
handwriting_set = os.listdir(handwriting_imgs_dir)
saved_path = "synthesis_imgs"
text_block_path = ""
make_dirs(saved_path) 


In [93]:
!ls

__pycache__              collect_chinese_word.txt utils
bg_img                   fonts                    word
cfg.py                   generate_data.ipynb      手写字体数据集
collect_chinese_word.py  synthesis_imgs


In [84]:
# punctuation = hanzi.punctuation#中文标点符号
zhon_collections = "collect_chinese_word.txt"
with open(zhon_collections, 'r') as f:
    zhon_chars = f.read(-1)
# print(zhon_chars)
char_nums = len(zhon_chars)
word_len = np.random.randint(5, 25)
start = 0
newCanvas = Image.new("RGB", (2512,2512), 'white')#新建背景画布

# newCanvas.show()
h_back, w_back = newCanvas.size
font_type = os.path.join("fonts", np.random.choice(os.listdir("fonts/")))
font_size = np.random.randint(15, 30)
margin_ratio = 1 / 8
texts = zhon_chars[start:start + word_len]

# ImageFont = ImageFont.truetype(font=font, size=np.random.randint(5,10)) 
font = ImageFont.truetype(font=font_type, size=font_size)#生成好了随机大小，类型的字体
x_start, y_start = int(margin_ratio * w_back), int(margin_ratio * h_back)
draw = ImageDraw.Draw(newCanvas)
w_txt, h_txt = draw.textsize(font=font, text=texts)
with Image.open("hand_sample.bmp") as hand_img:
    w_hand, h_hand = hand_img.size
    hw_ratio = h_hand / w_hand
    draw.text(xy=(x_start, y_start), text=texts, font=font, fill=(255,0,0))
    hand_img.resize((int(hw_ratio * h_back * 1/5), int(w_back * 1/5)))
    newCanvas.paste(hand_img, (x_start + w_txt, y_start + h_txt) )
    newCanvas.show()

In [ ]:
def make_thumb(path, sizes=(75, 32, 16)):
    base, ext = os.path.splitext(path)
    try:
        im = Image.open(path)
    except IOError:
        return 
    mode = im.mode
    if mode not in  ('L', 'RGB'):
        if mode == "RGBA":
            alpha = im.split()[3]
            bgmask = alpha.point(lambda x: 255-x)
            im = im.convert()

## TODO
* 使用PIL实现在图像上写一部分文字
** 读取背景图片
** 读取文字库txt，然后从中选取一部分区块文字，然后进行生成图片
** 选择文字区块，长度随机(1-40),字体大小要随机(字体大小要与背景图像尺寸相关)，字体类型随机，
* 读取手写字体并贴到打印字体的合成图片上(取最大外界矩形，判断一下文字的区间   )
* 打印体和手写体都需要确定location，并且形成标注文件


确定两头，字库不重复，保证每个字出现在开头结尾，判断每个字是否出现在头和尾

In [93]:
round(123 / 11, 3)#小数点后取三位
123 // 22 #地板除
123 / 22 #float除法
min(12, 34, 43)#取最小的
max(34, 253, 45)#取最大的
np.random.uniform(0,10)

0.7753738378480579

In [ ]:
def get_text_block():
    char_nums = np.random.randint(5, 20)
    

In [39]:
random.seed(666)

TL = glob.glob(handwriting_imgs_dir+'T*')#glob.glob()返回正则匹配到的结果，return一个结果list
blur_ratio = 0.1
text_block_path = "collect_chinese_word.txt"#文字路径
fonts = os.listdir(fonts_dir)
with open(text_block_path, "r") as f:
    items = f.readlines()
    
bg_imgs = os.listdir(background_imgs_dir)
sample = Image.open(os.path.join(background_imgs_dir, np.random.choice(bg_imgs)))
# sample.show()
print(sample.size)
bg_h, bg_w = sample.size
new_one = Image.new('RGB', (bg_h, bg_w), 'white')
font_type = os.path.join(fonts_dir, random.choice(fonts))
scale_ratio = 1 / 40
base_size = int(bg_h * scale_ratio)
font_size = random.randint(0, 10) + base_size

blank = Image.new("RGB", [1024, 768], "white")
drawObj = ImageDraw.Draw(sample)
# drawObj.line([100, 100, 100, 600], fill='red')
# drawObj.line([200,200,200,500], fill='black')
text = "敢问路在何方"
font1 = ImageFont.truetype(font=font_type, size=font_size)#生成好了随机大小，类型的字体
# drawObj.rectangle([0, 200, 300, 400])
bbs_list = []
for i in range(bg_h // font_size):
    random_interval = random.randint(0, font_size)
    if random_interval > bg_h:
        break
    h_loc = i * font_size
    bbs_list.append(ia.BoundingBox(0, h_loc, w_txt, h_loc + h_txt))
    w_txt, h_txt = drawObj.textsize(text=text, font=font1)

    drawObj.rectangle([0, h_loc, w_txt, h_loc + h_txt], outline=(255, 7, 4) )
#     print(w_txt, h_txt)
    drawObj.text([0, h_loc], text, 'black', font=font1)
    if np.random.randint(0, 10) < 8:
        blank = Image.new('RGB', (w_txt, h_txt), 'red')
        sample.paste(blank, (w_txt, h_loc + h_txt))
# bbs = ia.BoundingBoxesOnImage(bbs_list, shape=sample.size)
# bbs.draw_on_image(np.array(sample), thickness=2, color=[0,5,255])
sample_blurred = motion_blur(sample, degree=np.random.randint(0,10), angle=np.random.randint(0,90))
Image.fromarray(sample_blurred).show()

NameError: name 'handwriting_imgs_dir' is not defined

In [ ]:
def gen_txt_per_row():
    h_back, w_back = bg_img.size
    hw_ratio = h_back / w_back
    margin_ratio = 1/4
    y_start = margin_ratio * h_back + np.random.randint(0, 10)
    x_start = margin_ratio * hw_ratio * w_back + np.random.randint(0, 10)

In [ ]:
def gen_single_char_text():
    msTime = int(round(time.time()))

In [ ]:
def gen_text(img, first_end_text, text_block, start_char, ds):
    draw = ImageDraw.Draw(img)
    w_

In [ ]:
def motion_blur(image, degree=12, angle=45):
    image = np.array(image)

    # 这里生成任意角度的运动模糊kernel的矩阵， degree越大，模糊程度越高
    M = cv2.getRotationMatrix2D((degree / 2, degree / 2), angle, 1)
    motion_blur_kernel = np.diag(np.ones(degree))
    motion_blur_kernel = cv2.warpAffine(motion_blur_kernel, M, (degree, degree))

    motion_blur_kernel = motion_blur_kernel / degree
    blurred = cv2.filter2D(image, -1, motion_blur_kernel)

    # convert to uint8
    cv2.normalize(blurred, blurred, 0, 255, cv2.NORM_MINMAX)
    blurred = np.array(blurred, dtype=np.uint8)
    return blurred
sample.show()